# Extracting and cleaning Pitchfork reviews data

The goal: gather as much information as plausible and possible on each individual review and place it in the corresponding row of the dataframe, and then export the dataframe as a CSV file.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

## The data is an sqlite database found here https://www.kaggle.com/nolanbconaway/pitchfork-data/data

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///database.sqlite')
engine.table_names()

['artists', 'content', 'genres', 'labels', 'reviews', 'years']

In [4]:
# I find syntax for INNER JOIN of multiple tables clunky, so I use an alternative, albeit a little antiquated syntax here
query = 'SELECT c.reviewid AS reviewid_main, * FROM content c, genres g, labels l, reviews r, years y \
        WHERE c.reviewid = g.reviewid AND g.reviewid = l.reviewid AND l.reviewid = r.reviewid AND r.reviewid = y.reviewid' 
with engine.connect() as con:
    result = con.execute(query)
    df = pd.DataFrame(result.fetchall())
    df.columns = result.keys()

In [5]:
df.head()

,reviewid_main,reviewid,content,reviewid,genre,reviewid,label,reviewid,title,artist,...,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,reviewid,year
0,22703,22703,"“Trip-hop” eventually became a ’90s punchline,...",22703,electronic,22703,virgin,22703,mezzanine,massive attack,...,0,nate patrin,contributor,2017-01-08,6,8,1,2017,22703,1998.0
1,22721,22721,"Eight years, five albums, and two EPs in, the ...",22721,metal,22721,hathenter,22721,prelapsarian,krallice,...,0,zoe camp,contributor,2017-01-07,5,7,1,2017,22721,2016.0
2,22659,22659,Minneapolis’ Uranium Club seem to revel in bei...,22659,rock,22659,fashionable idiots,22659,all of them naturals,uranium club,...,0,david glickman,contributor,2017-01-07,5,7,1,2017,22659,2016.0
3,22659,22659,Minneapolis’ Uranium Club seem to revel in bei...,22659,rock,22659,static shock,22659,all of them naturals,uranium club,...,0,david glickman,contributor,2017-01-07,5,7,1,2017,22659,2016.0
4,22661,22661,Kleenex began with a crash. It transpired one ...,22661,rock,22661,kill rock stars,22661,first songs,"kleenex, liliput",...,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,22661,2016.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26018 entries, 0 to 26017
Data columns (total 22 columns):
reviewid_main     26018 non-null int64
reviewid          26018 non-null int64
content           26018 non-null object
reviewid          26018 non-null int64
genre             23317 non-null object
reviewid          26018 non-null int64
label             25977 non-null object
reviewid          26018 non-null int64
title             26018 non-null object
artist            26018 non-null object
url               26018 non-null object
score             26018 non-null float64
best_new_music    26018 non-null int64
author            26018 non-null object
author_type       20340 non-null object
pub_date          26018 non-null object
pub_weekday       26018 non-null int64
pub_day           26018 non-null int64
pub_month         26018 non-null int64
pub_year          26018 non-null int64
reviewid          26018 non-null int64
year              25426 non-null float64
dtypes: float64(2), 

### Let's delete 'reviewid' columns that duplicate our 'reviewid_main' column

In [7]:
df.drop(['reviewid'], axis =1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26018 entries, 0 to 26017
Data columns (total 17 columns):
reviewid_main     26018 non-null int64
content           26018 non-null object
genre             23317 non-null object
label             25977 non-null object
title             26018 non-null object
artist            26018 non-null object
url               26018 non-null object
score             26018 non-null float64
best_new_music    26018 non-null int64
author            26018 non-null object
author_type       20340 non-null object
pub_date          26018 non-null object
pub_weekday       26018 non-null int64
pub_day           26018 non-null int64
pub_month         26018 non-null int64
pub_year          26018 non-null int64
year              25426 non-null float64
dtypes: float64(2), int64(6), object(9)
memory usage: 3.4+ MB


In [8]:
# Time to wrangle that dataframe, and for that we need to see how many missing and duplicative values it's got
import pandas_profiling
pandas_profiling.ProfileReport(df)

C:\Users\Dinara\Anaconda3\lib\site-packages\matplotlib\__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Number of variables,17
Number of observations,26018
Total Missing (%),2.0%
Total size in memory,3.4 MiB
Average record size in memory,136.0 B
Numeric,6
Categorical,9
Boolean,1
Date,0
Text (Unique),0
Rejected,1


### As seen above, pandas profiling report says that we have 186 duplicate rows. Let's eliminate duplicates, keeping one copy of each unique row. 

In [9]:
df.drop_duplicates(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25832 entries, 0 to 26017
Data columns (total 17 columns):
reviewid_main     25832 non-null int64
content           25832 non-null object
genre             23224 non-null object
label             25791 non-null object
title             25832 non-null object
artist            25832 non-null object
url               25832 non-null object
score             25832 non-null float64
best_new_music    25832 non-null int64
author            25832 non-null object
author_type       20247 non-null object
pub_date          25832 non-null object
pub_weekday       25832 non-null int64
pub_day           25832 non-null int64
pub_month         25832 non-null int64
pub_year          25832 non-null int64
year              25240 non-null float64
dtypes: float64(2), int64(6), object(9)
memory usage: 3.5+ MB


### There are 18,389 unique values in column 'reviewid_main' and the same number of unique URLs, so that is the final number of entries (i.e. rows) we are looking to achieve. If we remove all rows with duplicative 'reviewid_main' and URL values we may lose some data on the columns that currently contain nulls in some rows. Those are: 'label', 'author_type', and 'year'. Let's then keep the rows with the least NULL values.

In [10]:
df_clean1 = (df.groupby(['reviewid_main', 'url'], as_index=False).apply(lambda x: x.loc[x.count(axis=1).idxmax()])).reset_index(drop = True)
df_clean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18389 entries, 0 to 18388
Data columns (total 17 columns):
reviewid_main     18389 non-null int64
content           18389 non-null object
genre             16024 non-null object
label             18351 non-null object
title             18389 non-null object
artist            18389 non-null object
url               18389 non-null object
score             18389 non-null float64
best_new_music    18389 non-null int64
author            18389 non-null object
author_type       14485 non-null object
pub_date          18389 non-null object
pub_weekday       18389 non-null int64
pub_day           18389 non-null int64
pub_month         18389 non-null int64
pub_year          18389 non-null int64
year              17983 non-null float64
dtypes: float64(2), int64(6), object(9)
memory usage: 2.4+ MB


In [11]:
df_clean1.head()

,reviewid_main,content,genre,label,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,year
0,1,"Aberfeldy recorded their debut, Young Forever,...",rock,rough trade,young forever,aberfeldy,http://pitchfork.com/reviews/albums/1-young-fo...,3.4,0,marc hogan,senior staff writer,2004-08-30,0,30,8,2004,2004.0
1,6,Can there be any purpose behind a master's deg...,electronic,silber,pure tone audiometry,aarktica,http://pitchfork.com/reviews/albums/6-pure-ton...,7.4,0,eric carr,contributor,2003-04-08,1,8,4,2003,2003.0
2,7,Funny what gets tagged for popular comebacks t...,electronic,darla,or you could just go through your whole life a...,aarktica,http://pitchfork.com/reviews/albums/7-or-you-c...,8.1,0,christopher dare,None,2002-05-16,3,16,5,2002,2002.0
3,8,Aarktica didn't seem to know what he was on 20...,electronic,darla,bleeding light,aarktica,http://pitchfork.com/reviews/albums/8-bleeding...,7.2,0,sam ubl,None,2005-02-06,6,6,2,2005,2005.0
4,10,ABCS are all about unity. Their bandname come...,jazz,troubleman unlimited,abcs,abcs,http://pitchfork.com/reviews/albums/10-abcs/,8.0,0,michael bernstein,None,2002-05-07,1,7,5,2002,2002.0


### Let's see how many non-null values we eliminated, keeping in mind that the 29% of values we eliminated in columns 'reviewid_main' and URL were duplicates.

In [12]:
print('Percentage decrease in non-null elements per column')
for col in df.columns:
    print (col+':', "{0:.2f}".format((df[col].count() - df_clean1[col].count())/df[col].count()))

Percentage decrease in non-null elements per column
reviewid_main: 0.29
content: 0.29
genre: 0.31
label: 0.29
title: 0.29
artist: 0.29
url: 0.29
score: 0.29
best_new_music: 0.29
author: 0.29
author_type: 0.28
pub_date: 0.29
pub_weekday: 0.29
pub_day: 0.29
pub_month: 0.29
pub_year: 0.29
year: 0.29


### Looks pretty good! We did not eliminate much more than 29% of items per column.

### Let's fix data type for 'pub_date', before we start looking for outliers etc.

In [13]:
df_clean1['pub_date'] = df_clean1['pub_date'].astype('datetime64[D]')

### Let's check out the data after cleaning to see if there are any outliers or unexpected values

In [14]:
pandas_profiling.ProfileReport(df_clean1)

Number of variables,17
Number of observations,18389
Total Missing (%),2.1%
Total size in memory,2.4 MiB
Average record size in memory,136.0 B
Numeric,6
Categorical,7
Boolean,1
Date,1
Text (Unique),1
Rejected,1


### There seem to be 10 reviews that don't contain any data (see Variables > 'content' in the pandas profiling report above), despite being non-null. There are no more anomalies in the data. Let's zoom in on those rows.

In [15]:
df_clean1[df_clean1['content']== '']

,reviewid_main,content,genre,label,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,year
5659,9033,,rock,barsuk,free to stay,smoosh,http://pitchfork.com/reviews/albums/9033-free-...,7.1,0,rachel khong,None,2006-06-21,2,21,6,2006,2006.0
6053,9464,,electronic,elektra,shine on,jet,http://pitchfork.com/reviews/albums/9464-shine...,0.0,0,ray suzuki,contributor,2006-10-02,0,2,10,2006,2006.0
6638,10072,,rock,rca,because of the times,kings of leon,http://pitchfork.com/reviews/albums/10072-beca...,5.4,0,roque strew,None,2007-04-05,3,5,4,2007,2007.0
7000,10450,,pop/r&b,sony,planet earth,prince,http://pitchfork.com/reviews/albums/10450-plan...,4.8,0,douglas wolk,contributor,2007-07-23,0,23,7,2007,2007.0
7047,10497,,rock,marmalade,we are ever so clean,blossom toes,http://pitchfork.com/reviews/albums/10497-we-a...,8.6,0,douglas wolk,contributor,2007-08-08,2,8,8,2007,1967.0
7157,10613,,electronic,fabric,fabric 34,ellen allien,http://pitchfork.com/reviews/albums/10613-fabr...,7.9,0,roque strew,None,2007-08-31,4,31,8,2007,2007.0
14704,18702,,experimental,anti-,aheym,"kronos quartet, bryce dessner",http://pitchfork.com/reviews/albums/18702-bryc...,7.9,0,jayson greene,senior editor,2013-11-08,4,8,11,2013,2013.0
14996,19018,,None,kanine,interrupt,bleeding rainbow,http://pitchfork.com/reviews/albums/19018-blee...,6.4,0,jayson greene,senior editor,2014-02-24,0,24,2,2014,2014.0
16220,20312,,experimental,buzz,sallows,anamai,http://pitchfork.com/reviews/albums/20312-sall...,7.1,0,melody lau,contributor,2015-03-12,3,12,3,2015,2015.0
17329,21584,,experimental,superior viaduct,four organs / phase patterns,steve reich,http://pitchfork.com/reviews/albums/21584-four...,7.6,0,jayson greene,senior editor,2016-05-25,2,25,5,2016,1971.0


### Since the text of reviews (i.e. 'content' column) is very important for our analysis, we are going to have to get rid of the entries where the review content is not available.

In [16]:
df_clean2 = df_clean1[df_clean1['content']!= '']
df_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18379 entries, 0 to 18388
Data columns (total 17 columns):
reviewid_main     18379 non-null int64
content           18379 non-null object
genre             16015 non-null object
label             18341 non-null object
title             18379 non-null object
artist            18379 non-null object
url               18379 non-null object
score             18379 non-null float64
best_new_music    18379 non-null int64
author            18379 non-null object
author_type       14478 non-null object
pub_date          18379 non-null datetime64[ns]
pub_weekday       18379 non-null int64
pub_day           18379 non-null int64
pub_month         18379 non-null int64
pub_year          18379 non-null int64
year              17973 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(6), object(8)
memory usage: 2.5+ MB


In [17]:
pandas_profiling.ProfileReport(df_clean2)

Number of variables,18
Number of observations,18379
Total Missing (%),2.0%
Total size in memory,2.5 MiB
Average record size in memory,144.0 B
Numeric,6
Categorical,7
Boolean,1
Date,1
Text (Unique),1
Rejected,2


### Finally, let's export the resulting dataframe into a CSV file for future use.

In [18]:
df_clean2.to_csv('pitchfork_data_cleaned_2018-03-29.csv')